# ADS2 - Assignment 1 - Data Handling and Processing with PySpark

**STUDENT NAME -** Mukesh Avudaiappan

**STUDENT ID -** 22024161


In this assignment, you will be analysing the popularity of films and TV shows on the streaming platform, Netflix. Using your knowledge of PySpark DataFrames and Spark SQL, you will produce a number of "downstream" data products to analyse trends in global streaming habits.

Download the dataset from this [Kaggle](https://www.kaggle.com/dhruvildave/netflix-top-10-tv-shows-and-films) page. A copy of the `all_weeks_countries.csv` file is also available on the canvas page for this assignment.

Your task is to load in the data and produce a number of "downstream" data products and plots as described below.

The PySpark installation and setup is provided below for conveinience.

IMPORTANT: DO NOT EDIT OR REMOVE THE COMMENT TAGS IN THE CODE CELLS

In [2]:
# CodeGrade Tag Init1

# Apache Spark uses Java, so first we must install that
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# CodeGrade Tag Init2
# Mount Google Drive and unpack Spark
from google.colab import drive
drive.mount('/content/drive')
!tar xzf /content/drive/MyDrive/spark-3.3.0-bin-hadoop3.tgz

Mounted at /content/drive


In [4]:
# CodeGrade Tag Init3
# Set up environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.0-bin-hadoop3"

In [5]:
# CodeGrade Tag Init4
# Install findspark, which helps python locate the pyspark module files
!pip install -q findspark
import findspark
findspark.init()

In [6]:
# Finally, we initialse a "SparkSession", which handles the computations
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [7]:
# Load the all_weeks_countries.csv into your Colab Notebook as a DataFrame.
netflixcsvpath = '/content/drive/MyDrive/all-weeks-countries.csv'

# Data is loaded with header: True and an inferred schema
netflixDF = (spark
           .read
           .option('header', 'True')
           .option('inferSchema', 'True')
           .csv(netflixcsvpath)
          )

In [8]:
# pyspark.sql.functions countains all the transformations and actions you will
# need
from pyspark.sql import functions as F

# Exercise 1 - Data Preparation


1.   Create two separate DataFrames for Films and TV.
2.   For the Films data, drop the column containing the season names.
3.   For the TV data, replace any null values in the season name column with the show name.

In [9]:
# CodeGrade Tag Ex1a

### Display the table and its schema

netflixDF.show()

netflixDF.printSchema()






+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                null|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                null|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                null|                         1|
|   Argentina|          AR|2022-08

In [10]:
# CodeGrade Tag Ex1b

### Seperate the data into two DataFrames for Films and TV
### Call the dataframes tvDF and filmsDF

filmsDF = netflixDF.filter(F.col("category") == "Films")
tvDF = netflixDF.filter(F.col("category") == "TV")



In [11]:
# CodeGrade Tag Ex1c

### Drop the 'season_title' column from the Films DataFrame, display the table
filmsDF = filmsDF.drop("season_title")
filmsDF.show()



+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          2|           Day Shift|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          3|Bank Robbers: The...|                         2|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          4|   The Next 365 Days|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          5|The Angry Birds M...|                         1|
|   Argentina|          AR|2022-08-21 00:00:00|   Films|        

In [12]:
# CodeGrade Tag Ex1d

### Use the F.isnull function to create a column showing where there are null
### values in the 'season_title' column. Replace the null values with the
### corresponding value from the 'show_title' column, then replace the
### 'season_title' column in the tvDF DataFrame.

# error in show title
tvDF = tvDF.withColumn("season_title", F.when(F.isnull(F.col("season_title")), F.col("show_title")).otherwise(F.col("season_title")))
tvDF.show()



+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|        season_title|cumulative_weeks_in_top_10|
+------------+------------+-------------------+--------+-----------+--------------------+--------------------+--------------------------+
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          1| Pasión de Gavilanes|Pasión de Gavilan...|                         5|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          2|        Another Self|Another Self: Sea...|                         4|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          3| Pasión de Gavilanes|Pasión de Gavilan...|                        58|
|   Argentina|          AR|2022-08-21 00:00:00|      TV|          4|            Manifest|  Manifest: Season 1|                         6|
|   Argentina|          AR|2022-08

# Exercise 2

Making use of the "groupBy" and "where" methods, find the number of weeks the show "Stranger Things" was in the Top 10 for the United Kingdom across all seasons. Store your result in a variable named "STWeeks"


In [13]:
# CodeGrade Tag Ex2
### Use the "where" method to create a new dataframe containing the data for
### the show Stranger Things in the Uniter Kingdom. Call this dataframe STDF.

STDF = netflixDF.where((F.col('show_title') == 'Stranger Things') & (F.col('country_name') == 'United Kingdom'))


### Using "groupBy" method and "F.count_distinct" function, find the total number of weeks
### Stranger Things spent in the top 10 of the UK, across all seasons. Show the
### result.


STWeeks = STDF.groupBy("show_title").agg(F.count_distinct("week").alias("total_weeks_in_top_10"))

STWeeks.show()



+---------------+---------------------+
|     show_title|total_weeks_in_top_10|
+---------------+---------------------+
|Stranger Things|                   13|
+---------------+---------------------+



# Exercise 3

Produce a dataframe containing only the Top 25 TV seasons in the UK, based on the number of weeks they spent in the Top 10.

In [15]:
# CodeGrade Tag Ex3
### Produce a dataframe containing the top 25 seasons by number of weeks in the
### top 10 of the United Kingdom, sorted by number of weeks. Store the dataframe
### in a variable called Top25


Top25 = (
    tvDF.filter(tvDF['country_name'] == 'United Kingdom').groupBy('season_title').agg(F.max('cumulative_weeks_in_top_10').alias('Total_Weeks')).orderBy('Total_Weeks', ascending=False).limit(25)
)

Top25.show(25)





+--------------------+-----------+
|        season_title|Total_Weeks|
+--------------------+-----------+
|   Stranger Things 4|         13|
|     Ozark: Season 4|         12|
|   Stranger Things 3|         12|
|   Stranger Things 2|         11|
|Squid Game: Season 1|         10|
|     Stranger Things|         10|
|Better Call Saul:...|         10|
|Bridgerton: Season 2|          9|
|Maid: Limited Series|          9|
|PAW Patrol: Season 6|          9|
|Inventing Anna: L...|          8|
| Money Heist: Part 5|          7|
|  Sex/Life: Season 1|          7|
|PIECES OF HER: Se...|          7|
|Clickbait: Limite...|          6|
|Sex Education: Se...|          6|
|Virgin River: Sea...|          6|
|Young Sheldon: : ...|          6|
|Young Sheldon: : ...|          6|
|The Lincoln Lawye...|          6|
|       You: Season 3|          6|
|PAW Patrol: Season 4|          6|
|The Umbrella Acad...|          5|
|Love Is Blind: Se...|          5|
|The Witcher: Seas...|          5|
+-------------------

# Exercise 4

For the show "Young Sheldon", find the country where each season spent the most time in the Top 10.

In [17]:
# CodeGrade Tag Ex4
### For each season of the show "Young Sheldon" find the countries where it spent
### the most time in the Top 10






young_sheldon_df = (
    netflixDF.filter(F.col('show_title') == 'Young Sheldon')
    .groupBy("season_title", "country_name")
    .agg(F.max("cumulative_weeks_in_top_10").alias("Top 10"))
    .orderBy( "Top 10",  ascending=False))




max_values_df = young_sheldon_df.groupBy("season_title").agg(F.max("Top 10").alias("max_value"))


young_sheldon_df = (young_sheldon_df.join(max_values_df,
(young_sheldon_df["season_title"] == max_values_df["season_title"])
& (young_sheldon_df["Top 10"] == max_values_df["max_value"]))
.drop(max_values_df["season_title"])
.drop(max_values_df["max_value"]))


young_sheldon_df = young_sheldon_df.orderBy("Top 10",  ascending=False).select("country_name", "season_title", "Top 10").orderBy("season_title")

young_sheldon_df.show(truncate=False)


+--------------+-------------------------+------+
|country_name  |season_title             |Top 10|
+--------------+-------------------------+------+
|United Kingdom|Young Sheldon: : Season 1|6     |
|Ireland       |Young Sheldon: : Season 1|6     |
|Canada        |Young Sheldon: : Season 1|6     |
|United Kingdom|Young Sheldon: : Season 2|6     |
|United Kingdom|Young Sheldon: : Season 3|4     |
|Australia     |Young Sheldon: Season 4  |1     |
|New Zealand   |Young Sheldon: Season 4  |1     |
+--------------+-------------------------+------+



# Exercise 5

For each country, find the film that spent the most time in the Top 10.

In [19]:
# CodeGrade Tag Ex5
### For each country, find the film that spent the most time in the Top 10
### Display the results in a Dataframe ordered by country name.


film_most_time_in_top_10 = (netflixDF.filter(F.col('Category') == 'Films')
    .select("Category", "show_title", "country_name", "cumulative_weeks_in_top_10")
    .groupBy("Category","show_title", "country_name")
    .agg(F.max("cumulative_weeks_in_top_10").alias("Top 10"))
    .orderBy( "Top 10",  ascending=False)

)

film_most_time_in_top_10 = film_most_time_in_top_10.orderBy('country_name')
film_most_time_in_top_10.show()


+--------+--------------------+------------+------+
|Category|          show_title|country_name|Top 10|
+--------+--------------------+------------+------+
|   Films|                Troy|   Argentina|     1|
|   Films|My Brother, My Si...|   Argentina|     1|
|   Films|        Street Kings|   Argentina|     1|
|   Films|     Against The Ice|   Argentina|     2|
|   Films|                Heat|   Argentina|     1|
|   Films|Todos queremos a ...|   Argentina|     1|
|   Films|   Company of Heroes|   Argentina|     1|
|   Films|       Blood Red Sky|   Argentina|     3|
|   Films|          Persuasion|   Argentina|     3|
|   Films|  The Dead Don't Die|   Argentina|     1|
|   Films|There's Someone I...|   Argentina|     2|
|   Films| Operation Mincemeat|   Argentina|     2|
|   Films|Fear Street Part ...|   Argentina|     3|
|   Films|       The Wasteland|   Argentina|     1|
|   Films|      The Four of Us|   Argentina|     1|
|   Films|      Redemption Day|   Argentina|     2|
|   Films|  

# Exercise 6

Calculate the number of weeks each film spent at the number 1 spot of each country's Top 10 list. Then find the films that spent the most time in the number 1 spot for each country.

In [20]:
# CodeGrade Tag Ex6a

### Create a column using the F.when function to calculate the number of weeks a
### films spends in the number 1 spot of the Top 10. Use the .otherwise method to
### set rows with no number 1 spots to zero. Use the .alias metod to call this
### column "weeks_at_1"


testy = netflixDF.filter(F.col("category") == "Films"  ).filter(F.col("weekly_rank") == 1  ).withColumn("weeks_at_1", F.when(filmsDF["weekly_rank"] == 1, 1).otherwise(0).alias("weeks_at_1"))
testy.show()


+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+----------+
|country_name|country_iso2|               week|category|weekly_rank|          show_title|season_title|cumulative_weeks_in_top_10|weeks_at_1|
+------------+------------+-------------------+--------+-----------+--------------------+------------+--------------------------+----------+
|   Argentina|          AR|2022-08-21 00:00:00|   Films|          1|      Look Both Ways|        null|                         1|         1|
|   Argentina|          AR|2022-08-14 00:00:00|   Films|          1|Bank Robbers: The...|        null|                         1|         1|
|   Argentina|          AR|2022-08-07 00:00:00|   Films|          1|       Purple Hearts|        null|                         2|         1|
|   Argentina|          AR|2022-07-31 00:00:00|   Films|          1|          Recurrence|        null|                         1|         1|
|   Argentina

In [24]:
# CodeGrade Tag Ex6b

### Group by country name and sow title, and use the .agg method and your new
### column to find the number of weeks each film spent in the top spot for each
### country.

result = testy.groupBy("country_name", "show_title").agg(F.sum("weeks_at_1").alias("weeks_at_1")).orderBy('weeks_at_1', ascending = False)
result.show()

+------------+-------------------+----------+
|country_name|         show_title|weeks_at_1|
+------------+-------------------+----------+
|      Russia|      Don't Look Up|         6|
|   Hong Kong|      Don't Look Up|         6|
|      Taiwan|      Don't Look Up|         5|
|      Taiwan|Gangubai Kathiawadi|         5|
|     Vietnam|         Red Notice|         5|
|      Israel|The Tinder Swindler|         4|
|     Estonia|      Don't Look Up|         4|
|  Bangladesh|    Haseen Dillruba|         4|
|     Ukraine|The Tinder Swindler|         4|
|     Vietnam|      Don't Look Up|         4|
|     Croatia|The Tinder Swindler|         4|
|    Bulgaria|         Red Notice|         4|
|       India|    Haseen Dillruba|         4|
|      Jordan|         Red Notice|         4|
|      Russia|The Tinder Swindler|         4|
|    Slovenia|      Don't Look Up|         4|
|    Pakistan|    Haseen Dillruba|         4|
|     Denmark|The Tinder Swindler|         4|
|     Estonia|The Tinder Swindler|

In [25]:
# CodeGrade Tag Ex6c

### Produce a dataframe grouped by country name that contains the show title and
### number of weeks at the number 1 spot of the top performing film in each
### country.


grouped_df = filmsDF.groupBy('country_name').agg(
    F.max('cumulative_weeks_in_top_10').alias('max_cumulative_weeks_in_top_10')
)


result_df = filmsDF.alias('a').join(
    grouped_df.alias('b'),
    (F.col('a.country_name') == F.col('b.country_name')) & (F.col('a.cumulative_weeks_in_top_10') == F.col('b.max_cumulative_weeks_in_top_10')),
    'inner'
).select('a.country_name', 'a.show_title', 'a.cumulative_weeks_in_top_10')


result_df.show()



+--------------+--------------------+--------------------------+
|  country_name|          show_title|cumulative_weeks_in_top_10|
+--------------+--------------------+--------------------------+
|     Argentina|  Sonic the Hedgehog|                         7|
|     Australia| Back to the Outback|                         8|
|       Austria|Harry Potter and ...|                         9|
|       Bahamas|  A Madea Homecoming|                         9|
|       Bahamas|A Madea Family Fu...|                         9|
|       Bahrain|          Red Notice|                         8|
|    Bangladesh|        Sooryavanshi|                        21|
|       Belgium|    The Adam Project|                         8|
|       Bolivia|               Shrek|                        18|
|        Brazil|Hotel Transylvani...|                         7|
|      Bulgaria|          Red Notice|                        15|
|        Canada|    The Adam Project|                         7|
|        Canada|How the G